In [1]:
!pip install pyspark


     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 37.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=3d6f03c68a86fecafd5615d7a04e86d2e12a98b5693c646ecd85d5fb61330e57
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
import pyspark
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
spark = SparkSession.builder.master("local[1]").appName("SparkApriori").getOrCreate()  
transaction_df = spark.read.csv("drive/MyDrive/BigData/Week3/store_data.csv")
transaction_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)



In [4]:
transaction_df.show()

+-----------------+------------+-------------+----------------+-------------+----------------+--------------+--------------+------------+--------------------+--------------+---------+-----+-----+-------------+------+-----------------+---------------+-------+---------+
|              _c0|         _c1|          _c2|             _c3|          _c4|             _c5|           _c6|           _c7|         _c8|                 _c9|          _c10|     _c11| _c12| _c13|         _c14|  _c15|             _c16|           _c17|   _c18|     _c19|
+-----------------+------------+-------------+----------------+-------------+----------------+--------------+--------------+------------+--------------------+--------------+---------+-----+-----+-------------+------+-----------------+---------------+-------+---------+
|           shrimp|     almonds|      avocado|  vegetables mix| green grapes|whole weat flour|          yams|cottage cheese|energy drink|        tomato juice|low fat yogurt|green tea|honey|sala

In [5]:
def remove_duplicate_columns(x):
    # Chiều dài của cột
    col_len = len(x)
    
    # RDD trống - tập hợp các giá trị
    columns = set()
    
    # Loại bỏ bất kỳ khoảng trắng bổ sung nào khỏi các phần tử và thêm các phần tử vào cột từ RDD 'x'
    for col in range(col_len):
        x_col = str(x[col]).strip()
        columns.add(x_col)
    
    # Để kiểm tra xem các phần tử có trong khung dữ liệu / RDD được cung cấp hay không
    if len(columns) < col_len:
        return []
    
    # Trả lại danh sách các mục đã được sắp xếp trong từng phần tử dưới dạng tuple
    return [(tuple(sorted(columns)))]

In [6]:
# Đối với tập dữ liệu đã cho, viết một hàm để trả về danh sách các mục riêng biệt trong tập dữ liệu

def generate_unique_item_set(df):
    # dataframe trống
    total_item_set_df = None
    
    # Lặp lại từng cột - 20 cột
    for col_index in range(20):
        
        # Đang tải các phần tử của từng cột riêng lẻ
        _c_df = df.select("_c" + str(col_index))
        
        if total_item_set_df is None:
            # Không có cho lần lặp đầu tiên trong vòng lặp
            total_item_set_df = _c_df
            
        else:
            # Sau lần lặp đầu tiên, nối các mục nhập từ mỗi cột thành total_item_set_df
            total_item_set_df = total_item_set_df.union(_c_df)
            
     # Giá trị trả về: Khung dữ liệu với các mục duy nhất (không lặp lại) và các giá trị rỗng bị xóa khỏi dataFrame
     # df.na cung cấp tất cả các giá trị null; tất cả các giá trị null phải được loại bỏ
     # .rdd chuyển đổi DataFrame thành RDD
     # remove_duplicate_columns phải được áp dụng cho các phần tử của RDD để mỗi mục trong giao dịch là một phần tử riêng biệt
     # Hãy nhớ rằng hàm 'remove_duplicate_columns' được áp dụng cho từng phần tử của RDD, nói tóm lại, mọi hàng của khung dữ liệu phải được chuyển vào đó.
    
    return total_item_set_df.select("_c0").na.drop().rdd.flatMap(remove_duplicate_columns).distinct().toDF()

In [7]:
item_sets = generate_unique_item_set(transaction_df)
item_sets.show()

+--------------------+
|                  _1|
+--------------------+
|   whole wheat pasta|
|           asparagus|
|            pancakes|
|         blueberries|
|            zucchini|
|              shrimp|
|             burgers|
|           spaghetti|
|         french wine|
|       strong cheese|
|extra dark chocolate|
|              melons|
|               cream|
|   frozen vegetables|
|           meatballs|
|          energy bar|
|            escalope|
|        energy drink|
|                mint|
|      vegetables mix|
+--------------------+
only showing top 20 rows



In [8]:
item_sets.count()

119

In [9]:
def filter_and_map_transaction(x, candidate_set_shared):
    
    c_k = []
    
    rows = len(candidate_set_shared.value)
    cols = len(candidate_set_shared.value[0])
    
    # Kiểm tra từng giao dịch
    for row_i in range(rows):
        item_set = set()
        for col_i in range(cols):
            item_set.add(candidate_set_shared.value[row_i][col_i])
        
        # Ánh xạ điều kiện với giá trị chính xác (1/0)
        if item_set.issubset(set(x)):
            c_k.append((candidate_set_shared.value[row_i], 1))
        else:
            c_k.append((candidate_set_shared.value[row_i], 0))
    return c_k


In [10]:
def get_all_possible_candidate_sets(candidate_item_sets_k, candidate_item_sets_0):
    
    # Chuyển đổi các phần tử của application_item_sets_k từ định dạng danh sách thành tuple
    # Bạn có thể tham khảo các lệnh dưới đây để kiểm tra lý do tại sao chỉ phần tử đầu tiên được sử dụng để chuyển đổi
    candidate_item_sets_k = candidate_item_sets_k.map(lambda x: tuple(x[0])).toDF()
    # toDF() chuyển đổi rdd thành dataFrame
    
    # Trả lại đơn hàng k + 1
    return candidate_item_sets_k.crossJoin(candidate_item_sets_0).rdd.flatMap(remove_duplicate_columns).distinct()
    # crossJoin sẽ giúp kết hợp một phần tử của một dataFrame với tất cả các phần tử của dataFrame khác

In [11]:
def get_freq_item_sets(total_records, candidate_sets_shared, transaction_df_rdd, min_support):
    filtered_item_set = transaction_df_rdd.flatMap(lambda x: filter_and_map_transaction(x, candidate_sets_shared)).reduceByKey(lambda a, b: a+b).map(lambda x: (x[0], x[1]/total_records)).filter(lambda x: x[1]>=min_support)
    return filtered_item_set

In [12]:
def is_freq_item_set_not_empty(freq_item_sets):
    return (freq_item_sets.count() > 0) and (freq_item_sets is not None)
    

In [13]:
def apriori(item_sets, transaction_df_rdd, min_support):
    total_records = transaction_df_rdd.count()
    freq_item_sets_all_orders = []
    candidate_sets_order_1 = spark.sparkContext.broadcast(item_sets.collect())
    frequent_item_sets_order_1 = get_freq_item_sets(total_records, candidate_sets_order_1, transaction_df_rdd, min_support)
    freq_item_sets_all_orders.append(frequent_item_sets_order_1)
    frequent_item_sets_order_1_df = frequent_item_sets_order_1.map(lambda x:tuple(x[0])).toDF()
    k = 0
    while is_freq_item_set_not_empty(freq_item_sets_all_orders[k]):
        current_candidate_sets = get_all_possible_candidate_sets(freq_item_sets_all_orders[k], frequent_item_sets_order_1_df)
        current_candidate_sets = spark.sparkContext.broadcast(current_candidate_sets.collect())
        current_frequent_item_sets = get_freq_item_sets(total_records, current_candidate_sets, transaction_df_rdd, min_support)
        freq_item_sets_all_orders.append(current_frequent_item_sets)
        k += 1
    return freq_item_sets_all_orders

In [14]:
transaction_df_rdd = transaction_df.rdd
freq_item_sets_all_orders = apriori(item_sets, transaction_df_rdd, 0.01)

print(freq_item_sets_all_orders)

[PythonRDD[177] at RDD at PythonRDD.scala:53, PythonRDD[178] at RDD at PythonRDD.scala:53, PythonRDD[179] at RDD at PythonRDD.scala:53, PythonRDD[180] at RDD at PythonRDD.scala:53]


In [15]:
freq_item_sets_all_orders[1].take(20)

[(('pancakes', 'shrimp'), 0.010531929076123183),
 (('burgers', 'pancakes'), 0.010531929076123183),
 (('pancakes', 'spaghetti'), 0.025196640447940274),
 (('frozen vegetables', 'pancakes'), 0.013464871350486601),
 (('chocolate', 'pancakes'), 0.01986401813091588),
 (('green tea', 'pancakes'), 0.01639781362485002),
 (('milk', 'pancakes'), 0.01653112918277563),
 (('eggs', 'pancakes'), 0.021730435941874418),
 (('mineral water', 'pancakes'), 0.03372883615517931),
 (('french fries', 'pancakes'), 0.020130649246767097),
 (('ground beef', 'pancakes'), 0.014531395813891481),
 (('cake', 'pancakes'), 0.011865084655379284),
 (('olive oil', 'pancakes'), 0.010798560191974404),
 (('shrimp', 'spaghetti'), 0.021197173710171976),
 (('frozen vegetables', 'shrimp'), 0.01666444474070124),
 (('chocolate', 'shrimp'), 0.017997600319957337),
 (('shrimp', 'tomatoes'), 0.011198506865751233),
 (('green tea', 'shrimp'), 0.011465137981602452),
 (('milk', 'shrimp'), 0.01759765364618051),
 (('eggs', 'shrimp'), 0.0141314